# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [154]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [155]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target, columns = ['price'])

In [156]:
X.head()

,CHAS,RM,DIS,B,LSTAT
0,0.0,6.575,0.542096,1.000000,-1.275260
1,0.0,6.421,0.623954,1.000000,-0.263711
2,0.0,7.185,0.623954,0.989737,-1.627858
3,0.0,6.998,0.707895,0.994276,-2.153192
4,0.0,7.147,0.707895,1.000000,-1.162114


In [157]:
y.head()

,price
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


## Train test split

Perform a train-test-split with a test set of 0.20.

In [158]:
from sklearn.model_selection import train_test_split

In [159]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)

In [160]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

404 102 404 102


In [161]:
from sklearn.linear_model import LinearRegression

Fit the model and apply the model to the make test set predictions

In [162]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_hat_test = linreg.predict(X_test)

Calculate the residuals and the mean squared error

In [163]:
from sklearn.metrics import mean_squared_error

In [164]:
residuals_test = y_hat_test - y_test
mse_test = mean_squared_error(y_hat_test,y_test)

In [165]:
mse_test

17.77255255535513

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [166]:
def kfolds(data, k):
    length = len(data)
    step = length / k
    big_step = int(step) + 1
    small_step = int(step)
    df = pd.DataFrame(data)
    folds = []
    starting_point = 0
    leftovers = length % k 
    for fold_ in range(1,k+1):
        if fold_ <= leftovers:
            fold = df[starting_point: starting_point + big_step]
            folds.append(fold)
            starting_point += big_step
        else:
            fold = df[starting_point: starting_point + small_step]
            folds.append(fold)
            starting_point += small_step
            
    # Force data as pandas dataframe
    # add 1 to fold size to account for leftovers           
    return folds

### Apply it to the Boston Housing Data

In [167]:
# Make sure to concatenate the data again

In [168]:
boston_data = pd.concat([X, y], axis = 1)

In [169]:
boston_data.head()

,CHAS,RM,DIS,B,LSTAT,price
0,0.0,6.575,0.542096,1.000000,-1.275260,24.0
1,0.0,6.421,0.623954,1.000000,-0.263711,21.6
2,0.0,7.185,0.623954,0.989737,-1.627858,34.7
3,0.0,6.998,0.707895,0.994276,-2.153192,33.4
4,0.0,7.147,0.707895,1.000000,-1.162114,36.2


In [170]:
boston_folds = kfolds(boston_data, 5)

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [184]:
train_errs = []
test_errs = []
features = X.columns
target = y.columns
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.DataFrame()
    for index in range(0,len(boston_folds)):
            if index != n:
                train = pd.concat([train, boston_folds[index]])
    test = boston_folds[n]
    # Fit a linear regression model
    linreg.fit(train[features], train[target])
    X_train = train.drop(['price'], axis = 1)
    X_test = test.drop(['price'], axis = 1)
    y_train = train['price']
    y_test = test['price']
    y_hat_train = linreg.predict(train[features])
    y_hat_test = linreg.predict(test[features])
    train_residuals = y_hat_train - np.array(y_train[target])
    test_residuals = y_hat_test - np.array(y_test[target])
    mse_train = mean_squared_error(y_hat_train,y_train)
    mse_test = mean_squared_error(y_hat_test,y_test)
    train_errs.append(mse_train)
    test_errs.append(mse_test)
    
    #Evaluate Train and Test Errors

print(train_errs)
print(test_errs)

[24.195577370388616, 23.032087348477972, 19.74507285797898, 15.31710138425119, 22.32997280754659]
[13.405144922008176, 17.444016800682032, 37.032711390020744, 58.27954384784233, 26.097988757148396]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [185]:
from sklearn.model_selection import cross_val_score

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [188]:
cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring="neg_mean_squared_error")

In [189]:
print(cv_5_results)

[-13.40514492 -17.4440168  -37.03271139 -58.27954385 -26.09798876]


##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!